# Natural Language Process - NLP

    + cleaning data
    + bag of words
    + text classification
    
    where is work:
    + predict genre of book
    + question -answer
    + summarization of book
    
    library:
    + nltk
    + spacy
    + standfor nlp
    + open nlp

In [1]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")

In [2]:
# import twitter data
data = pd.read_csv("nlp_csv/gender_classifier_nlp.csv",encoding="latin1")
data = pd.concat([data.gender,data.description],axis=1)

In [3]:
data.head()

,gender,description
0,male,i sing my own rhythm.
1,male,I'm the author of novels filled with family dr...
2,male,louis whining and squealing and all
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...


In [4]:
data.dropna(axis=0,inplace=True)
data.gender = [1 if each == "female" else 0 for each in data.gender]  # female = 0 , male = 1
data.head()

,gender,description
0,0,i sing my own rhythm.
1,0,I'm the author of novels filled with family dr...
2,0,louis whining and squealing and all
3,0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
4,1,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...


In [5]:
# cleaning data
# regular expression Re
import re
first_description = data.description[4]
print("ilk tanım : ",first_description,"\n")
description = re.sub("[^a-zA-Z]"," ",first_description) # a-z ye alma bunlar az olmayanlar yerine " " koy
print("harf olmayan karaker kaldırma : ",description,"\n")
description = description.lower() # buyuk harften kucuk harfe cevirme
print("harfleri küçültme : ",description)

ilk tanım :  Ricky Wilson The Best FRONTMAN/Kaiser Chiefs The Best BAND Xxxx Thank you Kaiser Chiefs for an incredible year of gigs and memories to cherish always :) Xxxxxxx 

harf olmayan karaker kaldırma :  Ricky Wilson The Best FRONTMAN Kaiser Chiefs The Best BAND Xxxx Thank you Kaiser Chiefs for an incredible year of gigs and memories to cherish always    Xxxxxxx 

harfleri küçültme :  ricky wilson the best frontman kaiser chiefs the best band xxxx thank you kaiser chiefs for an incredible year of gigs and memories to cherish always    xxxxxxx


In [10]:
# stopwords (irrelavent words) gereksiz kelimeler
import nltk
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords

#description = description.split()
description = nltk.word_tokenize(description)


[nltk_data] Downloading package stopwords to /home/ea/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ea/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


TypeError: expected string or bytes-like object

In [11]:
# gereksiz kelimeleri cikar
description = [word for word in description if not word in set(stopwords.words("english"))]

In [12]:
description

['ricky',
 'wilson',
 'best',
 'frontman',
 'kaiser',
 'chiefs',
 'best',
 'band',
 'xxxx',
 'thank',
 'kaiser',
 'chiefs',
 'incredible',
 'year',
 'gigs',
 'memories',
 'cherish',
 'always',
 'xxxxxxx']

In [13]:
# lemmatization loved => love , gitmeyecegim => git
import nltk as nlp
lemma = nlp.WordNetLemmatizer()
description = [lemma.lemmatize(word) for word in description]

In [15]:
description = " ".join(description)

In [16]:
description

'ricky wilson best frontman kaiser chief best band xxxx thank kaiser chief incredible year gig memory cherish always xxxxxxx'

In [17]:
# DATA CLEANING
description_list = []

for description in data.description:
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()
    description = nltk.word_tokenize(description)
    description = [word for word in description if not word in set(stopwords.words("english"))]
    lemma = nlp.WordNetLemmatizer()
    description = [lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description)

In [33]:
# BAG OF WORDS
from sklearn.feature_extraction.text import CountVectorizer # bag of words yaratmak icin kullanılan metod
max_features = 5000

count_vectorizer = CountVectorizer(max_features=max_features,stop_words="english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()

In [34]:
print("en sık kullanılan 5000 kelimeler : ",count_vectorizer.get_feature_names())

en sık kullanılan 5000 kelimeler :  ['aa', 'aaron', 'ab', 'abacus', 'abc', 'ability', 'able', 'absolute', 'absolutely', 'absurdity', 'abuse', 'ac', 'academia', 'academic', 'academy', 'acc', 'accept', 'accepted', 'access', 'accessory', 'accident', 'account', 'accountant', 'accounting', 'ace', 'achieve', 'act', 'acting', 'action', 'active', 'activist', 'activity', 'actor', 'actress', 'actual', 'actually', 'ad', 'adalah', 'adam', 'add', 'added', 'addict', 'addicted', 'addiction', 'addition', 'address', 'admin', 'administrator', 'admirer', 'adopted', 'adoption', 'adorable', 'adore', 'adult', 'advance', 'advanced', 'advancing', 'advantage', 'adventure', 'adventurer', 'adventurous', 'advertising', 'advice', 'advise', 'adviser', 'advisor', 'advisory', 'advocacy', 'advocate', 'advocating', 'aerial', 'aerospace', 'aesthetic', 'af', 'afc', 'affair', 'affiliate', 'affiliated', 'affiliation', 'affordable', 'aficionado', 'afraid', 'africa', 'african', 'afrikaner', 'afro', 'afternoon', 'ag', 'age', 

In [35]:
y = data.iloc[:,0].values  # male or female classes
x = sparce_matrix

In [36]:
# train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.1, random_state=42)

In [37]:
# naive bayes
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train,y_train)
# prediction
y_pred = nb.predict(x_test).reshape(-1,1)
print("accuracy: ",nb.score(y_pred,y_test))

accuracy:  0.48120764017252005
